In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
import lightgbm
from sklearn.metrics import mean_squared_error
import warnings
import gc
import 
warnings.filterwarnings('ignore')
np.random.seed(4590)

SyntaxError: invalid syntax (<ipython-input-1-6f99b6aafcd4>, line 10)

In [3]:
sales_train = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/test.csv')
train = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/train_total.csv')

In [4]:
item_categories = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/item_categories.csv')

items = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/items.csv')

shops = pd.read_csv('/Users/sinsakuokazaki/Project/pred-future-sales/\
input/competitive-data-science-predict-future-sales/shops.csv')

In [5]:
items_categories = pd.merge(items, item_categories, how = "outer", on = 'item_category_id')

In [6]:
vectorizer = CountVectorizer(encoding='koi8r')

In [7]:
item_name_vec = vectorizer.fit_transform(items_categories['item_name'])

In [8]:
vectorizer_cat = CountVectorizer(encoding='koi8r')
item_cat_vec = vectorizer_cat.fit_transform(items_categories['item_category_name'])

In [9]:
vectorizer_shop = CountVectorizer(encoding='koi8r')
shop_vec = vectorizer_shop.fit_transform(shops['shop_name'])

In [10]:
# #train_item_vec = vectorizer.fit_transform(train['item_name'])
# test_item_vec = vectorizer.fit_transform(test['item_name'])
# sales_item_vec = vectorizer.fit_transform(sales_train['item_name'])
# #train_cat_vec = vectorizer_cat.fit_transform(train['item_category_name'])
# test_cat_vec = vectorizer_cat.fit_transform(test['item_category_name'])
# sales_cat_vec = vectorizer_cat.fit_transform(sales_train['item_category_name'])
# #train_shop_vec = vectorizer_shop.fit_transform(train['shop_name'])
# test_shop_vec = vectorizer_shop.fit_transform(test['shop_name'])
# sales_shop_vec = vectorizer_shop.fit_transform(sales_train['shop_name'])
# # item_vec_train = train_item_vec.toarray()
# # cat_vec_train = train_cat_vec.toarray()
# # shop_vec_train = train_shop_vec.toarray()
# item_vec_test = test_item_vec.toarray()
# cat_vec_test = test_cat_vec.toarray()
# shop_vec_test = test_shop_vec.toarray()
# item_vec_sales = sales_item_vec.toarray()
# cat_vec_sales = sales_cat_vec.toarray()
# shop_vec_sales = sales_shop_vec.toarray()

In [11]:
params = {"max_depth": 7,
         "min_child_weight": 15,
         'objective':'regression',
         'max_depth': -1,
         'learning_rate': 0.01,
         "min_child_samples": 20,
         "boosting": "gbdt",
         "feature_fraction": 0.9,
         "bagging_freq": 1,
         "bagging_fraction": 0.9 ,
         "bagging_seed": 11,
         "metric": 'rmse',
         "lambda_l1": 0.1,
         "verbosity": -1,
         "nthread": 4,
         "random_state": 4590}

In [ ]:
month = train.columns[2:36]

for fold, i in enumerate(range(0, len(month)-1)):
    print('fold {}'.format(fold))
    
    item_vec_tra = vectorizer.transform(train[train[month[i]].notnull()]['item_name']) 
    item_arr_tra = item_vec_tra.toarray()
    cat_vec_tra = vectorizer_cat.transform(train[train[month[i]].notnull()]['item_category_name']) 
    cat_arr_tra = cat_vec_tra.toarray()
    shop_vec_tra = vectorizer_shop.transform(train[train[month[i]].notnull()]['shop_name']) 
    shop_arr_tra = shop_vec_tra.toarray()
    ids_tra = train[train[month[i]].notnull()][['shop_id', 'item_id']].values
    X_cur = np.concatenate((ids_tra, item_arr_tra, cat_arr_tra, shop_arr_tra), axis=1)
    Y_cur = train[train[month[i]].notnull()][month[i]].values
    del item_vec_tra, cat_vec_tra, shop_vec_tra, ids_tra, \
        item_arr_tra, cat_arr_tra, shop_arr_tra
        
    if fold == 0:
        X_tra = np.empty((0, X_cur.shape[1]))
        Y_tra = np.array([])
    
    X_tra = np.vstack((X_tra, X_cur))
    Y_tra = np.append(Y_tra, Y_cur)
    del X_cur, Y_cur
    
    item_vec_val = vectorizer.transform(train[train[month[i+1]].notnull()]['item_name'])
    item_arr_val = item_vec_val.toarray()
    cat_vec_val = vectorizer_cat.transform(train[train[month[i+1]].notnull()]['item_category_name'])
    cat_arr_val = cat_vec_val.toarray()
    shop_vec_val = vectorizer_shop.transform(train[train[month[i+1]].notnull()]['shop_name'])
    shop_arr_val = shop_vec_val.toarray()
    ids_val = train[train[month[i+1]].notnull()][['shop_id', 'item_id']].values
    
    X_val = np.concatenate((ids_val, item_arr_val, cat_arr_val, shop_arr_val), axis=1)
    Y_val = train[train[month[i+1]].notnull()][month[i+1]].values
    del item_vec_val, cat_vec_val, shop_vec_val, ids_val, \
        item_arr_val, cat_arr_val, shop_arr_val
    
    tra_data = lightgbm.Dataset(X_tra, label=Y_tra)
    val_data = lightgbm.Dataset(X_val, label=Y_val)
    
    model = lightgbm.train(params, tra_data, num_boost_round=1000, \
                           valid_sets=[tra_data, val_data],  verbose_eval=100, \
                           early_stopping_rounds = 100)
    pred = model.predict(X_val, num_iteration=model.best_iteration)
    
    np.sqrt(mean_squared_error(pred, Y_val))
    
    
    del X_val, Y_val, tra_data, val_data, model, pred

fold 0
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3107.52	valid_1's rmse: 3855.03
[200]	training's rmse: 2844.53	valid_1's rmse: 3778.25
[300]	training's rmse: 2726.26	valid_1's rmse: 3765.18
[400]	training's rmse: 2651.51	valid_1's rmse: 3755.22
[500]	training's rmse: 2594.95	valid_1's rmse: 3748.33
[600]	training's rmse: 2552.18	valid_1's rmse: 3744.74
[700]	training's rmse: 2514.87	valid_1's rmse: 3742.68
[800]	training's rmse: 2484.1	valid_1's rmse: 3739.43
[900]	training's rmse: 2454.98	valid_1's rmse: 3736.46
[1000]	training's rmse: 2430.1	valid_1's rmse: 3733.71
Did not meet early stopping. Best iteration is:
[1000]	training's rmse: 2430.1	valid_1's rmse: 3733.71
fold 1
Training until validation scores don't improve for 100 rounds.
[100]	training's rmse: 3303.71	valid_1's rmse: 4742.52
[200]	training's rmse: 3037.59	valid_1's rmse: 4732.18
Early stopping, best iteration is:
[173]	training's rmse: 3088.03	valid_1's rmse: 4726.21
fold 2
T